In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import pandas as pd
from modules import Preprocess, LGBFactory
from UserParams import Params
from datetime import datetime
# 最大列の表示数を指定
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 30)

## 前処理＆データ結合

In [2]:
# 集計開始年月日
_year = 2007

In [3]:
# インスタンス生成
preprocess = Preprocess()
# 各テーブルの前処理
manager = preprocess.preprocess_rawdata(since_=datetime(_year, 1, 1))
columns_info = manager.get_columns()

In [4]:
# データ内容の確認
_results = manager.__dict__['_DataManager__results']
_raceinfo = manager.__dict__['_DataManager__raceinfo']
_horseresults = manager.__dict__['_DataManager__horseresults']
_paybacks = manager.__dict__['_DataManager__paybacks']
_peds = manager.__dict__['_DataManager__peds']

In [5]:
_results.head()

,枠番,馬番,斤量,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank
race_id,,,,,,,,,,,
201305020802,3,5,54.0,2010102059,00422,牝,3,434.0,6.0,14,1
201305020802,1,2,56.0,2010101504,05386,牡,3,472.0,-2.0,14,2
201305020802,2,4,56.0,2010102416,01122,牡,3,428.0,2.0,14,3
201305020802,2,3,56.0,2010102120,00660,牡,3,498.0,0.0,14,4
201305020802,5,10,54.0,2010106575,01115,牝,3,458.0,-2.0,14,5


In [6]:
_raceinfo.head()

,開催,天気,馬場,日付,race_type,around,race_class,course_len
race_id,,,,,,,,
202006030602,中山,曇,稍重,2020-04-12,ダート,右,未勝利,12.0
201807030712,中京,晴,良,2018-07-21,芝,左,unknown,14.0
200907010303,中京,晴,良,2009-01-17,ダート,左,unknown,17.0
201305020802,東京,晴,重,2013-05-12,ダート,左,未勝利,13.0
201208020403,京都,晴,良,2012-02-05,ダート,右,新馬,12.0


In [7]:
_paybacks.head(11)

,type,matched,price
race_id,,,
200701010101,ワイド,1-4,590
200701010101,単勝,4,470
200701010101,枠連,4-8,320
200701010101,複勝,4,130
200701010101,複勝,11,100
200701010101,三連複,1-4-11,1290
200701010101,複勝,1,220
200701010101,馬単,4→11,1050
200701010101,馬連,4-11,340


In [8]:
_horseresults.head()

,日付,開催,天気,R,頭数,枠番,馬番,斤量,馬場,着順,着差,上り,賞金,weight,diff,race_type,course_len,first_corner,final_corner,mean_corner
horse_id,,,,,,,,,,,,,,,,,,,,
2011100102,2016-12-30,大井,晴,10,16,7,14,56.0,重,1,0.0,39.7,1300.0,498.0,2.0,ダート,16.0,3,2,2.50
2011100102,2016-11-03,川崎,晴,9,13,1,1,55.0,重,3,1.1,40.4,820.0,496.0,-3.0,ダート,16.0,7,4,5.00
2011100102,2016-10-06,川崎,晴,10,13,7,11,55.0,良,1,-0.9,39.2,400.0,499.0,3.0,ダート,15.0,4,1,2.75
2011100102,2016-09-12,川崎,曇,11,9,6,6,55.0,稍重,1,-1.4,39.1,400.0,496.0,-2.0,ダート,15.0,2,1,1.50
2011100102,2016-07-27,川崎,曇,11,12,8,12,53.0,良,1,0.0,38.8,600.0,498.0,6.0,ダート,16.0,3,1,2.00


In [9]:
_peds.head()

,peds_0,peds_1,peds_16
horse_id,,,
2011102707,2002107011,000a00033a,1992110220
2010104274,1995104427,000a0000cc,000a0062e2
2010102258,000a011099,000a00fdb3,000a01101c
2005106162,1996102442,1985103538,000a00012a
2007106238,1999107052,000a00035e,1990104462


In [10]:
columns_info.horseresults.T.set_axis(['対応するcolumn名の意味'], axis='columns')

,対応するcolumn名の意味
description,馬の過去成績テーブル
DATE,日付
PLACE,開催
WEATHER,天気
R,R
N_HORSES,頭数
FRAME_NUMBER,枠番
HORSE_NUMBER,馬番
WEIGHT,斤量
GROUND_STATE,馬場


In [11]:
# 「馬の過去成績」を対象としたデータ結合
horseresults = columns_info.horseresults
target_columns = [horseresults[key][0] for key in Params.TARGET_COLUMNS]
group_columns =  [horseresults[key][0] for key in Params.GROUP_COLUMNS]
merged_data = manager.merge(target_columns, group_columns)

In [12]:
merged_data.df.head()

,枠番,馬番,斤量,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank,開催,天気,馬場,日付,race_type,around,race_class,course_len,peds_0,peds_1,peds_16,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean
race_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200701010107,6,7,53.0,2004104659,01102,牡,3,460.0,4.0,11,1,札幌,晴,稍重,2007-08-11,ダート,右,unknown,10.0,000a000013,000a001607,000a009232,6.000000,1.100000,110.454545,2.818182,2.909091,2.863636,2.750000,2.750000,2.750000,0.450000,4.750000,193.750000,1.625000,1.625000,1.625000,0.812500,4.500000,129.375000,0.0,0.0,0.00,0.0,0.0,0.0,37.400000,37.637500,0.0,36.6
200701010107,5,5,55.0,2003105149,01093,牝,4,470.0,0.0,11,2,札幌,晴,稍重,2007-08-11,ダート,右,unknown,10.0,000a000d6b,000a001beb,000a0092cf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
200701010107,3,3,52.0,2003102358,01109,牝,4,482.0,0.0,11,3,札幌,晴,稍重,2007-08-11,ダート,右,unknown,10.0,000a000134,000a001607,000a0094a9,5.555556,0.688889,125.888889,8.000000,7.444444,7.722222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.444444,8.000000,7.722222,0.688889,5.555556,125.888889,0.0,0.0,0.00,0.0,0.0,0.0,37.700000,37.700000,0.0,0.0
200803030504,3,5,52.0,2003102358,01118,牝,5,498.0,2.0,15,8,福島,晴,良,2008-11-08,ダート,右,unknown,11.5,000a000134,000a001607,000a0094a9,6.047619,0.800000,91.285714,7.047619,6.714286,6.880952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.714286,7.047619,6.880952,0.800000,6.047619,91.285714,0.0,0.0,0.00,0.0,0.0,0.0,37.380952,37.380952,0.0,0.0
200907010603,1,1,52.0,2003102358,01118,牝,6,496.0,2.0,15,3,中京,晴,稍重,2009-01-25,ダート,左,unknown,10.0,000a000134,000a001607,000a0094a9,6.166667,0.800000,84.458333,7.375000,7.000000,7.187500,5.571429,5.714286,5.642857,0.585714,5.428571,69.142857,7.000000,7.375000,7.187500,0.800000,6.166667,84.458333,7.0,7.5,7.25,0.6,6.5,55.0,37.333333,37.333333,36.4,36.2


In [13]:
merged_data.df.columns

Index(['枠番', '馬番', '斤量', 'horse_id', 'jockey_id', 'sex', 'age', 'weight',
       'diff', 'n_horse', 'rank', '開催', '天気', '馬場', '日付', 'race_type',
       'around', 'race_class', 'course_len', 'peds_0', 'peds_1', 'peds_16',
       '着順_mean', '着差_mean', '賞金_mean', 'first_corner_mean',
       'final_corner_mean', 'mean_corner_mean', 'course_lenfinal_corner_mean',
       'course_lenfirst_corner_mean', 'course_lenmean_corner_mean',
       'course_len着差_mean', 'course_len着順_mean', 'course_len賞金_mean',
       'race_typefinal_corner_mean', 'race_typefirst_corner_mean',
       'race_typemean_corner_mean', 'race_type着差_mean', 'race_type着順_mean',
       'race_type賞金_mean', '開催final_corner_mean', '開催first_corner_mean',
       '開催mean_corner_mean', '開催着差_mean', '開催着順_mean', '開催賞金_mean', '上り_mean',
       'race_type上り_mean', '開催上り_mean', 'course_len上り_mean'],
      dtype='object')

In [14]:
merged_data.df.describe()

,枠番,馬番,斤量,age,weight,diff,n_horse,rank,course_len,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean
count,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000,54817.000000
mean,4.758250,7.961070,55.738767,3.945035,473.161246,0.387161,13.804057,7.346681,18.332743,1.686783,0.308980,38.974798,1.721612,1.656624,1.692666,1.029845,1.090791,1.064957,0.193309,0.980485,35.902661,1.441971,1.519186,1.484597,0.329438,1.407397,44.294708,1.256966,1.310575,1.286739,0.233145,1.259616,34.705889,8.289028,6.721141,5.960248,5.094472
std,2.259904,4.461225,2.272363,1.535277,29.632697,6.643080,2.190122,4.166122,6.367595,3.140632,0.672767,154.249494,3.266624,3.129685,3.196162,2.680729,2.829259,2.747879,0.710329,2.538824,216.768427,2.968047,3.129145,3.043216,0.944285,2.863042,177.414508,2.984226,3.102155,3.031795,0.691546,2.965772,196.762077,15.180959,13.770164,13.298801,12.488538
min,1.000000,1.000000,48.000000,2.000000,334.000000,-40.000000,4.000000,1.000000,10.000000,0.000000,-2.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.300000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,54.000000,3.000000,454.000000,-4.000000,13.000000,4.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,8.000000,56.000000,3.000000,472.000000,0.000000,14.000000,7.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,12.000000,57.000000,5.000000,492.000000,4.000000,15.000000,11.000000,20.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.000000,18.000000,65.000000,13.000000,640.000000,48.000000,18.000000,18.000000,42.600000,18.000000,13.200000,6932.165000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,21.850000,18.000000,12690.400000,18.000000,18.000000,18.000000,21.500000,18.000000,6932.165000,18.000000,18.000000,18.000000,16.400000,18.000000,11894.000000,46.100000,46.600000,51.500000,52.500000


In [15]:
table_df = merged_data.table
table_df

,description,FRAME_NUMBER,HORSE_NUMBER,WEIGHT,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank,PLACE,WEATHER,GROUND_STATE,DATE,race_type,around,race_class,course_len,peds_0,peds_1,peds_16,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean
0,結合後テーブル,枠番,馬番,斤量,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank,開催,天気,馬場,日付,race_type,around,race_class,course_len,peds_0,peds_1,peds_16,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean


In [16]:
# ラベルエンコーディング＆ダミー変数化の処理による特徴量生成
functions = {
    'dumminize':        {'columns': []},
    'label_encoding':   {'columns': [table_df[key][0] for key in Params.CATEGORY_COLUMNS]},
    'encode_jockey_id': {},
    'encode_horse_id':  {},
    'drop':             {'columns': table_df['DATE'].to_list()},
}
inputs = manager.get_features(merged_data.df, functions)

In [17]:
inputs.features

,枠番,馬番,斤量,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank,開催,天気,馬場,race_type,around,race_class,course_len,peds_0,peds_1,peds_16,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean
race_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200701010107,6,7,53.0,4679,156,2,3,460.0,4.0,11,1,6,2,1,0,0,3,10.0,1,184,460,6.000000,1.100000,110.454545,2.818182,2.909091,2.863636,2.750000,2.750000,2.750000,0.450000,4.750000,193.750000,1.625000,1.625000,1.625000,0.812500,4.500000,129.375000,0.0,0.0,0.00,0.0,0.0,0.0,37.400000,37.637500,0.0,36.6
200701010107,5,5,55.0,3355,148,1,4,470.0,0.0,11,2,6,2,1,0,0,3,10.0,141,241,467,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
200701010107,3,3,52.0,2973,163,1,4,482.0,0.0,11,3,6,2,1,0,0,3,10.0,45,184,482,5.555556,0.688889,125.888889,8.000000,7.444444,7.722222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.444444,8.000000,7.722222,0.688889,5.555556,125.888889,0.0,0.0,0.00,0.0,0.0,0.0,37.700000,37.700000,0.0,0.0
200803030504,3,5,52.0,2973,172,1,5,498.0,2.0,15,8,8,2,2,0,0,3,11.5,45,184,482,6.047619,0.800000,91.285714,7.047619,6.714286,6.880952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.714286,7.047619,6.880952,0.800000,6.047619,91.285714,0.0,0.0,0.00,0.0,0.0,0.0,37.380952,37.380952,0.0,0.0
200907010603,1,1,52.0,2973,172,1,6,496.0,2.0,15,3,0,2,1,0,1,3,10.0,45,184,482,6.166667,0.800000,84.458333,7.375000,7.000000,7.187500,5.571429,5.714286,5.642857,0.585714,5.428571,69.142857,7.000000,7.375000,7.187500,0.800000,6.166667,84.458333,7.0,7.5,7.25,0.6,6.5,55.0,37.333333,37.333333,36.4,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310010609,6,10,56.0,28341,198,2,3,482.0,-2.0,13,5,4,3,3,0,0,0,17.0,2254,254,806,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
202310010609,3,3,56.0,28097,404,2,3,480.0,14.0,13,7,4,3,3,0,0,0,17.0,24377,2331,1119,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
202310010609,7,11,54.0,28124,239,1,3,462.0,-8.0,13,10,4,3,3,0,0,0,17.0,981,750,975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [18]:
inputs.features.columns

Index(['枠番', '馬番', '斤量', 'horse_id', 'jockey_id', 'sex', 'age', 'weight',
       'diff', 'n_horse', 'rank', '開催', '天気', '馬場', 'race_type', 'around',
       'race_class', 'course_len', 'peds_0', 'peds_1', 'peds_16', '着順_mean',
       '着差_mean', '賞金_mean', 'first_corner_mean', 'final_corner_mean',
       'mean_corner_mean', 'course_lenfinal_corner_mean',
       'course_lenfirst_corner_mean', 'course_lenmean_corner_mean',
       'course_len着差_mean', 'course_len着順_mean', 'course_len賞金_mean',
       'race_typefinal_corner_mean', 'race_typefirst_corner_mean',
       'race_typemean_corner_mean', 'race_type着差_mean', 'race_type着順_mean',
       'race_type賞金_mean', '開催final_corner_mean', '開催first_corner_mean',
       '開催mean_corner_mean', '開催着差_mean', '開催着順_mean', '開催賞金_mean', '上り_mean',
       'race_type上り_mean', '開催上り_mean', 'course_len上り_mean'],
      dtype='object')

## 学習

In [19]:
# インスタンス作成
factory = LGBFactory()
lgb = factory.create(inputs.features)

In [20]:
_lgb_manager = lgb.__dict__['_LightGBM__manager']
_train_size = len(_lgb_manager.train_dataset.data)
_valid_size = len(_lgb_manager.validation_dataset.data)
_test_size = len(_lgb_manager.test_dataset.data)
print(f'train size:      {_train_size} (≒ {int(_train_size / 6)}レース分)')
print(f'validation size: {_valid_size} (≒ {int(_valid_size / 6)}レース分)')
print(f'test size:       {_test_size}  (≒ {int(_test_size / 6)}レース分)')

train size:      17280 (≒ 2880レース分)
validation size: 3825 (≒ 637レース分)
test size:       3812  (≒ 635レース分)


In [21]:
best_params = lgb.train_with_tuning()

min_data_in_leaf, val_score: 0.683406: 100%|##########| 5/5 [00:04<00:00,  1.24it/s]


In [22]:
best_params

,best_params
objective,binary
metric,binary_logloss
boosting,gbdt
verbose,-1
seed,3
feature_pre_filter,False
lambda_l1,0.0
lambda_l2,0.0
num_leaves,31
feature_fraction,0.7


In [23]:
lgb.test()

,precision,recall,f1-score,support
other,0.514480,0.611290,0.558722,1860.000000
Top3,0.548689,0.450307,0.494654,1952.000000
accuracy,0.528856,0.528856,0.528856,0.528856
macro avg,0.531584,0.530799,0.526688,3812.000000
weighted avg,0.531997,0.528856,0.525915,3812.000000


In [24]:
importance = lgb.feature_importance()
importance.head(30)

,features,importance
4,jockey_id,89
17,peds_0,58
19,peds_16,48
7,weight,27
18,peds_1,22
36,race_type着順_mean,20
10,開催,16
32,race_typefinal_corner_mean,11
16,course_len,11
6,age,10


In [25]:
_ = factory.save(lgb, version=f'model-{_year}-2022')